In [1]:
import pandas as pd
import math
import snappy
import sage

In [2]:
pd.set_option('display.max_colwidth', None)

# Table of Knot Invariants for $T(4,n)$
Knot signature, Alexander polynomial, Arf, upsilon, etc.

### TODO - implement lower bounds from Binns, et al. (https://arxiv.org/pdf/2109.09187)

### Symmetric Alexander Polynomial

In [3]:
def symmetric_alexander_poly(K):
    alex = K.alexander_polynomial(norm=False)
    alex_sym = alex.shift(-min(alex.exponents()) // 2) 
    return alex_sym

In [4]:
T45 = snappy.Link('T(4,5)')
T45.alexander_polynomial()

t^12 - t^11 + t^8 - t^6 + t^4 - t + 1

In [5]:
symmetric_alexander_poly(T45)

t^-6 - t^-5 + t^-2 - 1 + t^2 - t^5 + t^6

### $d$-invariant for $T(p,q)$

See Proposition 3.8 in Fairchild, et al.

In [6]:
def d_invariant(T_pq):
    alex_sym = symmetric_alexander_poly(T_pq)
    coeff_dict = alex_sym.dict()
    pos_exponents = [k for k in coeff_dict.keys() if k>0]
    
    d_T = 0
    for j in pos_exponents:
        d_T += j * coeff_dict[j]
        
    return 2 * d_T

### Make the table for arbitrary $(p,q)$

In [7]:
def make_table(p, max_q=None):
    if max_q is None:
        max_q = p+12
        
    invariants = []
    for q in range(p+1, max_q+1):
        if math.gcd(p,q) != 1: continue
    
        knot_name = f'T({p},{q})'
        knot = snappy.Link(knot_name)
        sage_knot = knot.sage_link()
    
        floer = knot.knot_floer_homology()
        #print(floer)
        
        invariants.append([q, knot_name, knot.signature(new_convention=False), sage_knot.arf_invariant(), floer['nu'], d_invariant(knot)])
        
    return pd.DataFrame(data=invariants, columns=['q', 'Knot', '$\\sigma$', 'Arf', '$\\nu$', 'd'])

### TODO - should $\sigma$ be negative?
See [signature documentation](https://snappy.computop.org/spherogram.html#spherogram.Link.signature) new_convention.

"SnapPy 3.0 switched the sign convention for the signature so that “positive knots have negative signatures”. You can recover the previous default by:" `L.signature(new_convention=False)`

In [8]:
df = make_table(4, max_q=20)
df

,q,Knot,$\sigma$,Arf,$\nu$,d
0,5,"T(4,5)",8,1,6,6
1,7,"T(4,7)",14,0,9,8
2,9,"T(4,9)",16,0,12,8
3,11,"T(4,11)",22,1,15,10
4,13,"T(4,13)",24,1,18,14
5,15,"T(4,15)",30,0,21,16
6,17,"T(4,17)",32,0,24,16
7,19,"T(4,19)",38,1,27,18
8,21,"T(4,21)",40,1,30,22
9,23,"T(4,23)",46,0,33,24


Observe that Remark 2.2 from Fairchild, et al. holds:

For odd $q$,
$$
\text{Arf}(T(p,q)) =
\begin{cases}
0 & p \text{ odd or } q\equiv\pm1\bmod 8, \\
1 & p \text { even and } q \equiv\pm 3\bmod 8.
\end{cases}
$$

### TODO - fix $d$-invariant implementation

Some data for verifying that `d_invariant` is working properly.

Check these against Proposition 3.8 from Fairchild, et al.

<div>
<center>
<img src="fairchild_prop_3-8.png" width="500"/>
</center>
</div>

In [9]:
make_table(5, max_q=11)

,q,Knot,$\sigma$,Arf,$\nu$,d
0,6,"T(5,6)",16,1,10,-6
1,7,"T(5,7)",16,0,12,-8
2,8,"T(5,8)",20,1,14,-10
3,9,"T(5,9)",24,0,16,-12
4,11,"T(5,11)",24,0,20,-12


$d(5,q)$ has correct residue class, but shouldn't be negative?

In [10]:
make_table(6, max_q=17)

,q,Knot,$\sigma$,Arf,$\nu$,d
0,7,"T(6,7)",18,0,15,12
1,11,"T(6,11)",34,1,25,18
2,13,"T(6,13)",36,1,30,18
3,17,"T(6,17)",52,0,40,24


$d(6,n)$ looks good!

# Lower bounds on $\gamma_4(T(4,n))$

Let's see for which $n$ does $\gamma_4(T(4,n))\ge 2$ using Proposition 2.3 from Fairchild, et al.

Proposition 2.3. Given a knot $K$ in $S^3$, if $$\sigma(K)+4\text{Arf}(K)\equiv 4\bmod 8$$ then $\gamma_4(K)\ge2$.

In [11]:
sig_arf = df['$\\sigma$'] + 4 * df['Arf']
sig_arf_lb_cond = sig_arf % 8
lb_applies = ['✓' if x == 4 else ' ' for x in sig_arf_lb_cond]
pd.DataFrame(data=zip(df['q'], sig_arf, sig_arf_lb_cond, lb_applies),
             columns=['q', '$\\sigma$/Arf expr', 'expr mod 8', 'Lower bound applies?'])
             #columns=['q', '$\\sigma(K)+4\\text{Arf}(K)$', '$\\sigma(K)+4\\text{Arf}(K)\\bmod8$', 'Lower bound applies?'])

,q,$\sigma$/Arf expr,expr mod 8,Lower bound applies?
0,5,12,4,✓
1,7,14,6,
2,9,16,0,
3,11,26,2,
4,13,28,4,✓
5,15,30,6,
6,17,32,0,
7,19,42,2,
8,21,44,4,✓
9,23,46,6,


### TODO - look into fixing column header latex formatting

https://stackoverflow.com/questions/72027874/python-formatting-a-pandas-dataframe-head-with-latex

Now let's compute the lower bound in Theorem 2.6, again from Fairchild, et al.

For a knot $K$ in $S^3$,
$$
\left| \nu(K)+\frac{\sigma(K)}{2} \right| \le \gamma_4(K).
$$

In [12]:
lb_nu = abs(df['$\\nu$'] + 0.5 * df['$\\sigma$'])

In [19]:
pd.DataFrame(data=zip(df['q'], lb_nu),
             columns=['q', '$\\nu$ lower bound'])
             #columns=['q', '$\\left| \\nu(K)+\\frac{\\sigma(K)}{2} \\right|$'])

#.style.set_table_styles([dict(selector="th", props=[('max-width', '120px'),
#                                ('text-overflow', 'ellipsis'), ('overflow', 'hidden')])])

,q,$\nu$ lower bound
0,5,10.0
1,7,16.0
2,9,20.0
3,11,26.0
4,13,30.0
5,15,36.0
6,17,40.0
7,19,46.0
8,21,50.0
9,23,56.0


Lastly, let's compute the lower bound from Theorem 2.7 using the $d$-invariant.

For a knot $K$,
$$
\frac{\sigma(K)}{2} - d(K) \le \gamma_4(K).
$$

In [14]:
lb_d = 0.5 * df['$\\sigma$'] - df['d']

In [15]:
pd.DataFrame(data=zip(df['q'], lb_d),
             columns=['q', '$d$ lower bound'])
             #columns=['q', '$\\frac{\\sigma(K)}{2} - d(K) \\le \\gamma_4(K)$'])

,q,$d$ lower bound
0,5,-2.0
1,7,-1.0
2,9,0.0
3,11,1.0
4,13,-2.0
5,15,-1.0
6,17,0.0
7,19,1.0
8,21,-2.0
9,23,-1.0


# Putting it all together

Computing the maximum lower bound according to the three given in Fairchild, et al.

In [16]:
# non-orientable 4-genus is always a positive integer (>= 1),
# so just give the trivial lower bound if the signature/arf lb doesn't apply
sig_arf_lb = [2 if x == 4 else 1 for x in sig_arf_lb_cond]

In [17]:
max_lbs = [max(x) for x in zip(sig_arf_lb, lb_nu, lb_d)]

In [18]:
pd.DataFrame(zip(df['q'], df['Knot'], sig_arf_lb, lb_nu, lb_d, max_lbs),
             columns=['q', 'Knot', 'arf lb', 'nu lb', 'd lb', 'maximum lb'])

,q,Knot,arf lb,nu lb,d lb,maximum lb
0,5,"T(4,5)",2,10.0,-2.0,10.0
1,7,"T(4,7)",1,16.0,-1.0,16.0
2,9,"T(4,9)",1,20.0,0.0,20.0
3,11,"T(4,11)",1,26.0,1.0,26.0
4,13,"T(4,13)",2,30.0,-2.0,30.0
5,15,"T(4,15)",1,36.0,-1.0,36.0
6,17,"T(4,17)",1,40.0,0.0,40.0
7,19,"T(4,19)",1,46.0,1.0,46.0
8,21,"T(4,21)",2,50.0,-2.0,50.0
9,23,"T(4,23)",1,56.0,-1.0,56.0
